In [4]:
import gc
import pandas as pd
import numpy as np
import os

In [5]:
path = "./data/raw/"

In [6]:
print('loading prior')
priors = pd.read_csv(path + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv(path + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(path + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(path + 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])

loading prior
loading train
loading orders
loading products


In [7]:
rate = 0.1

ordersの中からrate分のtrain/testのデータを抽出。

In [8]:
orders[orders['eval_set']=='train'].shape

(131209, 7)

In [9]:
orders[orders['eval_set']=='test'].shape

(75000, 7)

In [10]:
orders_train_small = orders[orders['eval_set']=='train'].sample(frac=rate)

In [11]:
orders_test_small = orders[orders['eval_set']=='test'].sample(frac=rate)

In [12]:
orders_train_small.shape

(13121, 7)

In [13]:
orders_test_small.shape

(7500, 7)

In [14]:
orders_small_train_test = pd.concat([orders_test_small, orders_train_small])

In [15]:
orders_small_train_test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1894202,1248499,113706,test,19,1,16,30.0
2871904,2765051,173283,test,7,5,12,14.0
2426354,2273231,145998,test,16,5,12,7.0
621206,95743,37466,test,6,3,13,18.0
782859,792859,47068,test,4,1,16,30.0


抽出したdataのuser分だけ、ordersから抽出。これでrate分のordersが完成。

In [16]:
orders = orders[orders['user_id'].isin(orders_small_train_test['user_id'].values)]

In [17]:
orders.reset_index(drop=True, inplace=True)

In [18]:
orders.shape

(342270, 7)

In [19]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2717275,5,prior,1,3,12,NaN
1,1909121,5,prior,2,0,16,11.0
2,2267326,5,prior,3,3,18,10.0
3,157374,5,prior,4,1,18,19.0
4,2196797,5,train,5,0,11,6.0


In [20]:
train = train[train['order_id'].isin(orders['order_id'].values)]

In [21]:
train.reset_index(drop=True, inplace=True)

In [22]:
train.shape

(138291, 4)

In [23]:
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,631,21137,1,1
1,631,27344,2,1
2,631,42265,3,1
3,631,12211,4,0
4,631,29740,5,0


In [24]:
priors = priors[priors['order_id'].isin(orders['order_id'].values)]

In [25]:
priors.reset_index(drop=True, inplace=True)

In [26]:
priors.shape

(3237175, 4)

In [27]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered
0,45,16349,1,1
1,53,24852,1,1
2,53,37646,2,1
3,53,47209,3,1
4,53,36606,4,0


CSVで出力したい場合は以下を実行

In [28]:
priors.to_csv(path + 'order_products__prior_small.csv', index=False)
train.to_csv(path + 'order_products__train_small.csv', index=False)
orders.to_csv(path + 'orders_small.csv', index=False)